In [ ]:
from sklearn import datasets
from sklearn.preprocessing import label_binarize
import numpy as np
import tensorflow as tf
diris = datasets.load_iris()
iris = np.column_stack([diris['data'], diris['target']])
np.random.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])
print(iris)
print(len(iris))
n_hidden_1 = 30
n_hidden_2 = 30 
n_input = 4 
n_classes = 3
learning_rate = 0.001
training_epochs = 30
batch_size = 20
display_step = 1
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
weights = {
    'a1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'a2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'a3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_classes]))
}
def multiperceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['a1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['a2']), biases['b2'])
    output = tf.matmul(layer_2, weights['a3']) + biases['b3']
    return output
logits = multiperceptron(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size,:4], target_multiclass[i:i+batch_size]
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,Y: batch_y})
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")
    pred = tf.nn.softmax(logits)  
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))